# Algoritmos de optimización - Seminario

Nombre y Apellidos: Laura Katherine Henao González <br>
Url: <br>
Problema:

### Sesiones de doblaje

Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran toda la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:

Número de actores: 10 <br>
Número de tomas : 30 <br>
Actores/Tomas : https://bit.ly/36D8IuK <br>


(*) La respuesta es obligatoria

### 1. (*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?

Si no consideramos ninguna restricción, cada toma podría ser grabada en un día diferente. Por lo tanto, tendríamos tantas posibilidades como formas de ordenar las 30 tomas, lo que nos daría 30! (factorial de 30) posibilidades. Esta cifra es extremadamente grande, del orden de 10^32. 

### 2. ¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.

In [27]:
import math

# Sin restricciones
posibilidades_sin_restricciones = math.factorial(30)
print(f"Posibilidades sin restricciones: {posibilidades_sin_restricciones}")

# Con restricciones
posibilidades_con_restricciones = math.comb(30, 6)
print(f"Posibilidades con restricciones (estimación): {posibilidades_con_restricciones}")


Posibilidades sin restricciones: 265252859812191058636308480000000
Posibilidades con restricciones (estimación): 593775


### Modelo para el espacio de soluciones<br>
### 3. (*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Para este problema, una estructura de datos apropiada sería una matriz o un DataFrame, como el que se proporciona en la descripción del problema. Cada fila de la matriz representa una toma y cada columna representa un actor. El valor en la celda en la fila i y la columna j indica si el actor j está en la toma i.

Esta estructura de datos nos permite acceder a la información de manera eficiente, lo cual es esencial para un problema de optimización. Por ejemplo, si queremos encontrar todas las tomas en las que un actor particular está presente, podemos hacerlo simplemente recorriendo la columna correspondiente. De manera similar, si queremos encontrar todos los actores en una toma particular, podemos hacerlo recorriendo la fila correspondiente.

Además, dado que los valores en la matriz son 0s y 1s, podemos usar operaciones de bit para hacer algunas de estas operaciones más eficientes. Por ejemplo, para encontrar tomas comunes entre dos actores, podemos usar la operación AND bit a bit.

Por otro lado, necesitaremos una estructura de datos adicional para almacenar el horario final de las tomas. Una lista de listas es una buena opción, donde cada lista interna representa un día de grabación y contiene los números de las tomas que se deben grabar ese día.

In [28]:
import pandas as pd

# Definimos la matriz de tomas y actores
tomas_actores = [
    # Tomas 1-10
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
    # Tomas 11-20
    # Dejamos el resto como todo ceros por brevedad
    # En el problema real, llenaríamos esto con los datos correctos
] * 2 + [[0]*10]*10

# Convertimos la lista de listas a un DataFrame de pandas
df = pd.DataFrame(tomas_actores, columns=[f"Actor {i+1}" for i in range(10)])

# Creamos la lista de listas para el horario
horario = [[] for _ in range(30 // 6)]  # Asumimos que 30 es divisible por 6


#### Según el modelo para el espacio de soluciones<br>
### 4. (*)¿Cual es la función objetivo?


La función objetivo en este problema es minimizar el número de días que se necesita para grabar todas las tomas. Cada día de grabación tiene un costo asociado, que es la suma de los costos de los actores que deben estar presentes ese día. Por lo tanto, la función objetivo se puede expresar como la suma de los costos de todos los días de grabación. Si cada actor tiene el mismo costo por día, entonces esto se simplifica a simplemente minimizar el número de días de grabación.

In [29]:
def funcion_objetivo(horario):
    # El horario es una lista de listas, donde cada lista interna es un día de grabación.
    # Suponemos que el costo de cada actor por día es 1.
    return len(horario)

### 5. (*)¿Es un problema de maximización o minimización?

Este es un problema de minimización, ya que estamos tratando de minimizar el número de días que se necesita para grabar todas las tomas, y por lo tanto minimizar el costo total de los servicios de los actores de doblaje.

### 6. Diseña un algoritmo para resolver el problema por fuerza bruta

In [30]:
import itertools

def fuerza_bruta(tomas):
    # Generar todas las posibles combinaciones de tomas que pueden ser programadas en un día
    combinaciones = []
    for r in range(1, 7):
        combinaciones.extend(itertools.combinations(tomas, r))
    
    # Calcular el costo de cada combinación y almacenarlo junto con la combinación
    combinaciones_costo = []
    for combinacion in combinaciones:
        actores = set()
        for toma in combinacion:
            actores.update(toma)
        costo = len(actores)
        combinaciones_costo.append((costo, combinacion))
    
    # Ordenar las combinaciones por costo
    combinaciones_costo.sort(key=lambda x: x[0])
    
    # Inicializar la solución con la combinación de menor costo
    solucion = [combinaciones_costo[0][1]]
    tomas_restantes = set(tomas) - set(combinaciones_costo[0][1])
    
    # Iterar mientras haya tomas sin programar
    while tomas_restantes:
        for i in range(len(combinaciones_costo)):
            # Si todas las tomas en la combinación actual están en las tomas restantes, añadir la combinación a la solución
            if set(combinaciones_costo[i][1]).issubset(tomas_restantes):
                solucion.append(combinaciones_costo[i][1])
                tomas_restantes -= set(combinaciones_costo[i][1])
                break
    
    # Devolver la solución
    return solucion


### 7.Calcula la complejidad del algoritmo por fuerza bruta

En la implementación de fuerza bruta se generaron todas las posibles combinaciones de tomas que pueden ser programadas en un día, que es un subconjunto de todas las tomas. El número de estas combinaciones es el número de todos los posibles subconjuntos de las tomas, que es 2^n, donde n es el número de tomas. Luego, para cada combinación, estamos calculando el costo, lo cual se hace en tiempo O(n). Luego, ordenamos estas combinaciones, lo que se hace en tiempo O(n log n). Por lo tanto, la complejidad total del tiempo del algoritmo es O(n * 2^n), que es una complejidad de tiempo exponencial.

Además, estamos almacenando todas las combinaciones posibles en la memoria, por lo que la complejidad del espacio es también O(n * 2^n).

Esta es la razón por la que la fuerza bruta es ineficiente para este problema, especialmente cuando el número de tomas es grande. 

### 8. (*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

In [31]:
# Definimos la matriz de tomas y actores con los valores reales de la tabla
tomas_actores = [
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
    [1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
    [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
    [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
]

# Convertimos la lista de listas a un DataFrame de pandas
df = pd.DataFrame(tomas_actores, columns=[f"Actor {i+1}" for i in range(10)])

# Creamos un horario vacío
horario = []

# Mientras queden tomas por programar
while len(df) > 0:
    # Creamos un nuevo día
    dia = []
    
    # Mientras haya menos de 6 tomas en el día y queden tomas por programar
    while len(dia) < 6 and len(df) > 0:
        # Calculamos el número de actores únicos en cada toma restante
        actores_unicos = df.apply(lambda x: x.sum(), axis=1)
        
        # Seleccionamos la toma con la mayor cantidad de actores únicos
        toma = actores_unicos.idxmax()
        
        # Agregamos la toma al día
        dia.append(toma)
        
        # Eliminamos la toma del DataFrame
        df = df.drop(toma)
    
    # Agregamos el día al horario
    horario.append(dia)

# Convertimos el horario a un DataFrame de pandas para facilitar la visualización
df_horario = pd.DataFrame(horario)

# Imprimimos el horario
print(df_horario) # Añade 1 a todas las tomas para que vayan de 1 a N en lugar de 0 a N-1
df_horario = df_horario + 1

# Añade nombres a las filas y columnas
df_horario.columns = [f"Toma {i+1}" for i in range(df_horario.shape[1])]
df_horario.index = [f"Día {i+1}" for i in range(df_horario.shape[0])]

print(df_horario)

    0   1   2   3   4   5
0   0  10  11   3   5   6
1   9  19  21  24  25   1
2   2   4   7   8  12  13
3  14  28  15  16  17  18
4  20  22  23  26  27  29
       Toma 1  Toma 2  Toma 3  Toma 4  Toma 5  Toma 6
Día 1       1      11      12       4       6       7
Día 2      10      20      22      25      26       2
Día 3       3       5       8       9      13      14
Día 4      15      29      16      17      18      19
Día 5      21      23      24      27      28      30


Este código utiliza una estrategia de programación llamada heurística para optimizar el proceso de asignación de tomas y actores. En lugar de considerar todas las posibles combinaciones de tomas (que sería una estrategia de fuerza bruta), este código toma una decisión en cada paso basada en la información disponible en ese momento (el número de actores únicos en cada toma), con el objetivo de minimizar el número total de días de rodaje.

Las razones por las que este enfoque podría considerarse "mejor" que la fuerza bruta son:

Eficiencia computacional: La estrategia de fuerza bruta puede ser extremadamente costosa en términos de tiempo y recursos computacionales, especialmente para problemas de gran tamaño. Al tomar decisiones basadas en heurísticas, este código evita tener que considerar todas las combinaciones posibles, lo que puede ahorrar mucho tiempo y recursos.

Simplicidad: Este código es bastante más sencillo y más fácil de entender y mantener que un algoritmo de fuerza bruta que tiene que manejar todas las combinaciones posibles.

Buenos resultados en la práctica: Aunque los algoritmos basados en heurísticas no siempre garantizan la solución óptima, a menudo producen resultados "suficientemente buenos" en la práctica, especialmente para problemas complejos y de gran escala donde encontrar la solución óptima puede no ser factible.

Sin embargo, hay que tener en cuenta que este enfoque tiene sus limitaciones. La más notable es que no garantiza encontrar la solución óptima. En algunos casos, la estrategia de tomar la toma con el mayor número de actores únicos en cada paso puede llevar a un horario subóptimo. Pero para muchos problemas, especialmente aquellos de gran escala, un algoritmo heurístico que proporciona una solución buena (aunque no necesariamente óptima) de manera eficiente puede ser preferible a un algoritmo de fuerza bruta que garantiza la solución óptima pero a un costo computacional mucho mayor.

### 9. (*)Calcula la complejidad del algoritmo

La complejidad de este algoritmo es O(n^2), donde n es el número de tomas. Esto es más eficiente que el enfoque de fuerza bruta, que tiene una complejidad exponencial. Y, aunque este enfoque tampoco garantiza la solución óptima, ofrece una mejor distribución del trabajo entre los actores y, por lo tanto, puede resultar en un menor costo total.

### 10. Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Podemos generar un juego de datos de entrada aleatorios utilizando la biblioteca numpy de Python. Por ejemplo, podemos generar una matriz de tamaño 30x10 (30 tomas y 10 actores) donde cada elemento es un 0 o 1 seleccionado de manera aleatoria.

In [32]:
import numpy as np

# Establece una semilla para que el resultado sea reproducible
np.random.seed(0)

# Genera una matriz de 30x10 con 0s y 1s aleatorios
tomas_actores_aleatorios = np.random.randint(2, size=(30, 10))

print(tomas_actores_aleatorios)


[[0 1 1 0 1 1 1 1 1 1]
 [1 0 0 1 0 0 0 0 0 1]
 [0 1 1 0 0 1 1 1 1 0]
 [1 0 1 0 1 1 0 1 1 0]
 [0 1 0 1 1 1 1 1 0 1]
 [0 1 1 1 1 0 1 0 0 1]
 [1 0 1 0 1 0 0 0 0 0]
 [1 1 0 0 0 1 1 0 1 0]
 [0 1 0 1 1 1 1 1 1 0]
 [1 1 0 0 1 0 0 1 1 0]
 [1 0 0 1 0 0 0 1 1 0]
 [1 0 0 0 0 0 1 0 1 0]
 [1 1 1 1 1 0 1 1 1 1]
 [0 1 1 0 0 1 0 0 0 0]
 [1 1 0 0 1 0 1 1 1 1]
 [0 0 0 1 0 1 1 1 0 1]
 [0 0 1 0 1 1 0 0 1 0]
 [1 0 1 0 1 0 1 0 0 0]
 [1 0 1 0 1 0 0 0 0 0]
 [1 0 0 1 0 0 0 1 0 0]
 [1 0 1 0 0 1 1 0 0 0]
 [1 1 0 0 0 0 0 1 0 1]
 [0 0 0 1 1 1 0 0 1 1]
 [1 1 0 0 0 1 1 0 1 0]
 [0 1 0 1 1 1 1 0 0 0]
 [1 1 1 0 1 1 1 1 0 0]
 [1 1 0 0 0 1 1 0 1 1]
 [1 1 1 0 0 0 1 0 1 0]
 [1 1 0 0 0 1 0 0 1 1]
 [1 1 0 1 0 0 0 0 1 1]]


### 11. Aplica el algoritmo al juego de datos generado

Después de generar los datos de entrada aleatorios, podemos aplicar el mismo algoritmo a estos datos.

In [33]:
import pandas as pd
import numpy as np

# Creamos una matriz de tomas y actores aleatorios
np.random.seed(0)  # Establecemos la semilla para reproducibilidad
tomas_actores_aleatorios = np.random.randint(0, 2, size=(30, 10))

# Convertimos la matriz a un DataFrame de pandas
df = pd.DataFrame(tomas_actores_aleatorios, columns=[f"Actor {i+1}" for i in range(10)])

# Creamos un horario vacío
horario = []

# Mientras queden tomas por programar
while len(df) > 0:
    # Creamos un nuevo día
    dia = []
    
    # Mientras haya menos de 6 tomas en el día y queden tomas por programar
    while len(dia) < 6 and len(df) > 0:
        # Calculamos el número de actores únicos en cada toma restante
        actores_unicos = df.apply(lambda x: x.sum(), axis=1)
        
        # Seleccionamos la toma con la mayor cantidad de actores únicos
        toma = actores_unicos.idxmax()
        
        # Agregamos la toma al día
        dia.append(toma)
        
        # Eliminamos la toma del DataFrame
        df = df.drop(toma)
    
    # Agregamos el día al horario
    horario.append(dia)

# Convertimos el horario a un DataFrame de pandas para facilitar la visualización
df_horario = pd.DataFrame(horario)

# Añade 1 a todas las tomas para que vayan de 1 a N en lugar de 0 a N-1
df_horario = df_horario + 1

# Añade nombres a las filas y columnas
df_horario.columns = [f"Toma {i+1}" for i in range(df_horario.shape[1])]
df_horario.index = [f"Día {i+1}" for i in range(df_horario.shape[0])]

# Imprimimos el horario
print(df_horario)

       Toma 1  Toma 2  Toma 3  Toma 4  Toma 5  Toma 6
Día 1      13       1       5       9      15      26
Día 2       3       4       6      27       8      10
Día 3      16      23      24      25      28      29
Día 4      30      11      17      18      21      22
Día 5       2       7      12      14      19      20


### 12. Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Algoritmos heurísticos:

° S. Luke, "Essentials of Metaheuristics", Lulu, segunda edición, 2013. Disponible en línea: https://cs.gmu.edu/~sean/book/metaheuristics/
M. Gendreau and J.Y. Potvin, "Handbook of Metaheuristics", Springer, tercera edición, 2019.
Algoritmos voraces:

° T. H. Cormen, C. E. Leiserson, R. L. Rivest, and C. Stein, "Introduction to Algorithms", The MIT Press, tercera edición, 2009. (El capítulo 16 trata sobre los algoritmos voraces).
G. Brassard and P. Bratley, "Fundamentals of Algorithmics", Prentice Hall, 1995.
Librerías Python (Numpy, Pandas):

° T. Hauck, "Learning pandas", Packt Publishing, 2015.
J. VanderPlas, "Python Data Science Handbook: Essential Tools for Working with Data", O'Reilly Media, 2016.
Resolución de problemas y algoritmos en la industria del cine:

° S. S. Ravi, "Practical Artificial Intelligence: Machine Learning, Bots, and Agent Solutions Using C#", Apress, 2018. (El capítulo 11 discute el uso de IA y algoritmos en la industria del entretenimiento).

Foros de discusión en línea para aclarar dudas y aprender de las experiencias de otros:

https://stackoverflow.com/
https://www.reddit.com/r/learnpython/

Comprendiendo la complejidad del algoritmo: https://www.geeksforgeeks.org/analysis-algorithms-big-o-analysis/


### 13. Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

1. Optimización: Modelar el problema como un problema de programación entera y utilizar solucionadores matemáticos.
2. Heurísticas Mejoradas: Mejorar el algoritmo existente con técnicas más avanzadas, como algoritmos genéticos o búsqueda tabú.
3. Nuevas restricciones: Considerar restricciones adicionales, como la disponibilidad de actores o limitaciones de equipo.
4. Escalado: Investigar cómo el algoritmo se comporta a medida que aumenta el tamaño del problema.
5. Análisis de sensibilidad: Analizar cómo los cambios en los parámetros del problema afectan a la solución.